<a href="https://colab.research.google.com/github/AbdulRehman2106/Python-Assignment-4/blob/main/Password_Generater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import secrets
import string
import time
import os
from cryptography.fernet import Fernet
from colorama import Fore, Style
import pyperclip
from datetime import datetime

def show_splash_screen():
    os.system('cls' if os.name == 'nt' else 'clear')
    title = """

  _      __    __                         ______           ___                                 __      _____                      __             __
 | | /| / /__ / /______  __ _  ___       /_  __/__        / _ \___ ____ ____    _____  _______/ /     / ___/__ ___  ___ _______ _/ /____  ____  / /
 | |/ |/ / -_) / __/ _ \/  ' \/ -_)       / / / _ \      / ___/ _ `(_-<(_-< |/|/ / _ \/ __/ _  /     / (_ / -_) _ \/ -_) __/ _ `/ __/ _ \/ __/ /_/  _ _
 |__/|__/\__/_/\__/\___/_/_/_/\__/       /_/  \___/     /_/   \_,_/___/___/__,__/\___/_/  \_,_/      \___/\__/_//_/\__/_/  \_,_/\__/\___/_/   (_)  (_|_)

                                                    🔒 PASSWORD  GENERATOR 🔒

                                                    Created By Abdul Rehman..
    """
    print(title)
    print("\n🔄 Loading", end="")
    for _ in range(3):
        time.sleep(0.5)
        print(".", end="", flush=True)
    time.sleep(1)
    print("\n")

def typing_effect(text, delay=0.02):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(delay)
    print()

def generate_key():
    return Fernet.generate_key()

def encrypt_password(password, key):
    cipher_suite = Fernet(key)
    encrypted_password = cipher_suite.encrypt(password.encode())
    return encrypted_password

def decrypt_password(encrypted_password, key):
    cipher_suite = Fernet(key)
    decrypted_password = cipher_suite.decrypt(encrypted_password).decode()
    return decrypted_password

def check_strength(password):
    length = len(password)
    has_upper = any(c.isupper() for c in password)
    has_lower = any(c.islower() for c in password)
    has_digit = any(c.isdigit() for c in password)
    has_symbol = any(c in string.punctuation for c in password)

    score = sum([has_upper, has_lower, has_digit, has_symbol])

    if length >= 12 and score == 4:
        strength = "Strong"
        color = Fore.GREEN
    elif length >= 8 and score >= 3:
        strength = "Moderate"
        color = Fore.YELLOW
    else:
        strength = "Weak"
        color = Fore.RED

    print(f"{color}{strength}{Style.RESET_ALL}")
    return strength

def generate_password(length, use_upper, use_lower, use_digits, use_symbols):
    characters = ''
    if use_upper:
        characters += string.ascii_uppercase
    if use_lower:
        characters += string.ascii_lowercase
    if use_digits:
        characters += string.digits
    if use_symbols:
        characters += string.punctuation

    if not characters:
        return "Error: No character types selected."

    return ''.join(secrets.choice(characters) for _ in range(length))

def save_to_history(password, key):
    encrypted_password = encrypt_password(password, key)
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("password_history.txt", "a") as file:
        file.write(f"{current_date} | {encrypted_password.decode()}\n")

def is_password_expired():
    expiration_days = 30
    current_time = time.time()
    last_saved_time = os.path.getmtime("password_history.txt") if os.path.exists("password_history.txt") else 0
    time_diff = (current_time - last_saved_time) / (60 * 60 * 24)
    return time_diff > expiration_days

def get_password_strength(password):
    strength = check_strength(password)
    if strength == "Strong":
        return Fore.GREEN + "Strong" + Style.RESET_ALL
    elif strength == "Moderate":
        return Fore.YELLOW + "Moderate" + Style.RESET_ALL
    else:
        return Fore.RED + "Weak" + Style.RESET_ALL

def show_password_history(key):
    if not os.path.exists("password_history.txt"):
        typing_effect("\n⚠️ No Password History Found.")
        return

    typing_effect("\n📜 Password History With Strength Indicators:")
    with open("password_history.txt", "r") as file:
        for line in file:
            date, encrypted_password = line.split(" | ")
            encrypted_password = encrypted_password.strip().encode()
            password = decrypt_password(encrypted_password, key)
            strength = get_password_strength(password)
            print(f"Date: {date} | Password: {password} | Strength: {strength}")

def search_history_by_strength(key, strength_filter):
    if not os.path.exists("password_history.txt"):
        typing_effect("\n⚠️ No Password History Found.")
        return

    typing_effect(f"\n🔍 Searching For {strength_filter} Passwords:")
    with open("password_history.txt", "r") as file:
        for line in file:
            date, encrypted_password = line.split(" | ")
            encrypted_password = encrypted_password.strip().encode()
            password = decrypt_password(encrypted_password, key)
            strength = get_password_strength(password)
            if strength_filter.lower() in strength.lower():
                print(f"Date: {date} | Password: {password} | Strength: {strength}")

def export_history_to_file():
    if not os.path.exists("password_history.txt"):
        typing_effect("\n⚠️ No Password History Found To Export.")
        return

    with open("password_history.txt", "r") as file:
        content = file.read()

    with open("password_history_export.txt", "w") as export_file:
        export_file.write(content)

    typing_effect("\n✅ Password History Exported To 'password_history_export.txt'.")

def main():
    key_file = "key.key"
    if not os.path.exists(key_file):
        key = generate_key()
        with open(key_file, "wb") as key_file_obj:
            key_file_obj.write(key)
    else:
        with open(key_file, "rb") as key_file_obj:
            key = key_file_obj.read()

    if is_password_expired():
        typing_effect("\n⚠️ Password History Expired! Please Reset Your Password History.")
        if os.path.exists("password_history.txt"):
            os.remove("password_history.txt")

    while True:
        typing_effect("\nWould You Like To:\n1. Generate A Random Password\n2. Enter Your Own Password\n3. View Password History\n4. Search Password History By Strength\n5. Export Password History\n6. Exit")
        option = input("\nChoose: (1/2/3/4/5/6): ")

        if option == '1':
            length = int(input("🔢 Enter Password Length: "))
            use_upper = input("\nInclude Uppercase Letters? (Y/N): ").lower() == 'y'
            use_lower = input("\nInclude Lowercase Letters? (Y/N): ").lower() == 'y'
            use_digits = input("\nInclude Digits? (Y/N): ").lower() == 'y'
            use_symbols = input("\nInclude Symbols? (Y/N): ").lower() == 'y'

            typing_effect("\n🔄 Generating Password...\n")
            password = generate_password(length, use_upper, use_lower, use_digits, use_symbols)
            typing_effect(f"\n🔐 Password: {password}")
            print("\n📊 Strength: ", end="")
            check_strength(password)
            save_to_history(password, key)
            pyperclip.copy(password)
            typing_effect("\n📋 Copied To Clipboard!")

        elif option == '2':
            password = input("\n🔒 Enter Your Custom Password: ")
            typing_effect(f"\n🔐 Your Password: {password}")
            print("\n📊 Strength: ", end="")
            check_strength(password)
            save_to_history(password, key)
            pyperclip.copy(password)
            typing_effect("\n📋 Copied To Clipboard!")

        elif option == '3':
            show_password_history(key)

        elif option == '4':
            strength_filter = input("\nEnter Strength To Filter By (Strong/Moderate/Weak): ")
            search_history_by_strength(key, strength_filter)

        elif option == '5':
            export_history_to_file()

        elif option == '6':
            typing_effect("\n👋 Exiting... Stay Secure, Abdul Rehman!")
            break

        else:
            typing_effect("\n⚠️ Invalid Option. Please Choose 1, 2, 3, 4, 5 or 6.")

if __name__ == "__main__":
    show_splash_screen()
    main()
